In [1]:
import json
import pandas as pd

## 1 - Reconstruction des données EIREL

In [2]:
# Chargement des données avec les bonnes colonnes
def load_eirel(filepath, config_path='data/bronze/reference/eirel_columns.json'):
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    colonnes = {int(k): v for k, v in config['fixed_columns'].items()}
    
    temp = pd.read_csv(filepath, sep=";", skiprows=2, header=None, usecols=[17], nrows=1)
    nb_listes = temp.iloc[0, 0]
    
    start_idx = config['dynamic_columns']['start_index']
    for i in range(1, nb_listes + 1):
        colonnes[start_idx + 2*(i-1)] = f"code_liste_{i}"
        colonnes[start_idx + 2*(i-1) + 1] = f"voix_liste_{i}"
    
    data = pd.read_csv(filepath, sep=";", skiprows=2, header=None)
    data.rename(columns=colonnes, inplace=True)
    
    return data

In [3]:
# Récupération des données EIREL avec les configurations précices
data = load_eirel("data/bronze/eirel/Eirel-testMunicipales-nov2025-1erTour.txt")
data.head()

,num_ligne,type_election,annee,tour,departement,commune,bureau_vote,indicateur,nb_inscrits,nb_abstentions,...,nb_exprimes,nb_listes,code_liste_1,voix_liste_1,code_liste_2,voix_liste_2,code_liste_3,voix_liste_3,code_liste_4,voix_liste_4
0,1,MN,2025,1,92,48,1,I,931,131,...,700,4,1,200,2,300,3,150,4,50
1,2,MN,2025,1,92,48,2,I,1023,173,...,825,4,1,50,2,300,3,475,4,0
2,3,MN,2025,1,92,48,3,I,987,237,...,730,4,1,100,2,200,3,300,4,130
3,4,MN,2025,1,92,48,4,I,1036,236,...,780,4,1,150,2,250,3,300,4,80
4,5,MN,2025,1,92,48,5,I,1006,256,...,700,4,1,100,2,200,3,300,4,100


In [4]:
# Aperçu des colonnes
data.columns

Index(['num_ligne', 'type_election', 'annee', 'tour', 'departement', 'commune',
       'bureau_vote', 'indicateur', 'nb_inscrits', 'nb_abstentions',
       'nb_votants', 'nb_emargements', 'nb_blanc', 'nb_nuls', 'nb_exprimes',
       'nb_listes', 'code_liste_1', 'voix_liste_1', 'code_liste_2',
       'voix_liste_2', 'code_liste_3', 'voix_liste_3', 'code_liste_4',
       'voix_liste_4'],
      dtype='str')

In [5]:
# Dimension du jeu de données
data.shape

(32, 24)

## 2 - Sélection des colonnes à intégrer pour constituer le jeu de données

In [6]:
with open("data/bronze/reference/columns_repartition.json") as config_cols:
    cols_used = json.load(config_cols)

In [7]:
data[cols_used["metrics_bdv"]].head()

,bureau_vote,type_election,annee,tour,commune,nb_inscrits,nb_abstentions,nb_votants,nb_emargements,nb_blanc,nb_nuls
0,1,MN,2025,1,48,931,131,800,800,50,50
1,2,MN,2025,1,48,1023,173,850,850,15,10
2,3,MN,2025,1,48,987,237,750,750,10,10
3,4,MN,2025,1,48,1036,236,800,800,10,10
4,5,MN,2025,1,48,1006,256,750,750,25,25


In [8]:
data[cols_used["metrics_bdv"]].info()

<class 'pandas.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   bureau_vote     32 non-null     int64
 1   type_election   32 non-null     str  
 2   annee           32 non-null     int64
 3   tour            32 non-null     int64
 4   commune         32 non-null     int64
 5   nb_inscrits     32 non-null     int64
 6   nb_abstentions  32 non-null     int64
 7   nb_votants      32 non-null     int64
 8   nb_emargements  32 non-null     int64
 9   nb_blanc        32 non-null     int64
 10  nb_nuls         32 non-null     int64
dtypes: int64(10), str(1)
memory usage: 2.9 KB


In [9]:
data[cols_used["candidats_voix"]].head()

,bureau_vote,code_liste_1,voix_liste_1,code_liste_2,voix_liste_2,code_liste_3,voix_liste_3,code_liste_4,voix_liste_4
0,1,1,200,2,300,3,150,4,50
1,2,1,50,2,300,3,475,4,0
2,3,1,100,2,200,3,300,4,130
3,4,1,150,2,250,3,300,4,80
4,5,1,100,2,200,3,300,4,100


In [10]:
data[cols_used["candidats_voix"]].info()

<class 'pandas.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   bureau_vote   32 non-null     int64
 1   code_liste_1  32 non-null     int64
 2   voix_liste_1  32 non-null     int64
 3   code_liste_2  32 non-null     int64
 4   voix_liste_2  32 non-null     int64
 5   code_liste_3  32 non-null     int64
 6   voix_liste_3  32 non-null     int64
 7   code_liste_4  32 non-null     int64
 8   voix_liste_4  32 non-null     int64
dtypes: int64(9)
memory usage: 2.4 KB


## 3 - Transformation de données
> Faire une jointure pour récupérer les noms de bureaux de vote

### 3.1 - Récupération des noms de bureau

In [11]:
# Données d'enrichissement --> 
bdv = pd.read_csv(
    "https://data.meudon.fr/api/explore/v2.1/catalog/datasets/bureaux-de-vote-electoraux-2026/exports/csv", 
    sep=";",
    usecols=["num_bureau", "nom_bureau"]
)

In [12]:
bdv.head()

,num_bureau,nom_bureau
0,19,Ecole maternelle La Ruche
1,17,Ecole maternelle Prévert
2,32,L'Avant-Seine
3,16,Ecole maternelle Jean de la Fontaine
4,20,Pôle intergénérations


In [13]:
bdv.info()

<class 'pandas.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_bureau  32 non-null     int64
 1   nom_bureau  32 non-null     str  
dtypes: int64(1), str(1)
memory usage: 644.0 bytes


In [14]:
nom_bureau = data.merge(
    bdv, 
    left_on="bureau_vote", 
    right_on="num_bureau", 
    how="left"
)

In [15]:
nom_bureau = nom_bureau.copy()
nom_bureau.head()

,num_ligne,type_election,annee,tour,departement,commune,bureau_vote,indicateur,nb_inscrits,nb_abstentions,...,code_liste_1,voix_liste_1,code_liste_2,voix_liste_2,code_liste_3,voix_liste_3,code_liste_4,voix_liste_4,num_bureau,nom_bureau
0,1,MN,2025,1,92,48,1,I,931,131,...,1,200,2,300,3,150,4,50,1,Mairie
1,2,MN,2025,1,92,48,2,I,1023,173,...,1,50,2,300,3,475,4,0,2,Club Séniors Lavoisier
2,3,MN,2025,1,92,48,3,I,987,237,...,1,100,2,200,3,300,4,130,3,Espace jeunesse Val Fleury
3,4,MN,2025,1,92,48,4,I,1036,236,...,1,150,2,250,3,300,4,80,4,Complexe sportif René Leduc
4,5,MN,2025,1,92,48,5,I,1006,256,...,1,100,2,200,3,300,4,100,5,Ecole élémentaire Jules Ferry


In [16]:
nom_bureau.num_bureau.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])

In [17]:
nan_counts_bureau = nom_bureau[cols_used["candidats_voix"]].isna().sum()
nan_counts_bureau[nan_counts_bureau > 0]

Series([], dtype: int64)

### 3.2 - Transformation pivot table pour récupération des candidats et nombres de voix

In [18]:
candidats_voix = data[cols_used["candidats_voix"]].head()
candidats_voix.head()

,bureau_vote,code_liste_1,voix_liste_1,code_liste_2,voix_liste_2,code_liste_3,voix_liste_3,code_liste_4,voix_liste_4
0,1,1,200,2,300,3,150,4,50
1,2,1,50,2,300,3,475,4,0
2,3,1,100,2,200,3,300,4,130
3,4,1,150,2,250,3,300,4,80
4,5,1,100,2,200,3,300,4,100


In [19]:
candidats_voix.columns

Index(['bureau_vote', 'code_liste_1', 'voix_liste_1', 'code_liste_2',
       'voix_liste_2', 'code_liste_3', 'voix_liste_3', 'code_liste_4',
       'voix_liste_4'],
      dtype='str')

In [20]:
candidats_voix.bureau_vote.unique()

array([1, 2, 3, 4, 5])

In [21]:
# Sélectionner uniquement les colonnes voix_liste_*
voix_cols = [col for col in candidats_voix.columns if col.startswith('voix_liste_')]

candidats_voix_long = candidats_voix.melt(
    id_vars='bureau_vote',
    value_vars=voix_cols,
    var_name='candidat',
    value_name='nb_voix'
)

# Extraire le numéro du candidat
# candidats_voix_long['candidat'] = candidats_voix_long['candidat'].str.extract(r'(\d+)').astype(int)
candidats_voix_long['candidat'] = candidats_voix_long['candidat']

candidats_voix_long.head()

,bureau_vote,candidat,nb_voix
0,1,voix_liste_1,200
1,2,voix_liste_1,50
2,3,voix_liste_1,100
3,4,voix_liste_1,150
4,5,voix_liste_1,100


In [22]:
candidats_voix_long.bureau_vote.unique()

array([1, 2, 3, 4, 5])

In [23]:
voix_cols

['voix_liste_1', 'voix_liste_2', 'voix_liste_3', 'voix_liste_4']

In [24]:
candidats_voix_long[candidats_voix_long["bureau_vote"] == 1]

,bureau_vote,candidat,nb_voix
0,1,voix_liste_1,200
5,1,voix_liste_2,300
10,1,voix_liste_3,150
15,1,voix_liste_4,50


In [25]:
candidats_voix_long[candidats_voix_long["bureau_vote"] == 2]

,bureau_vote,candidat,nb_voix
1,2,voix_liste_1,50
6,2,voix_liste_2,300
11,2,voix_liste_3,475
16,2,voix_liste_4,0


In [26]:
candidats_voix_long[candidats_voix_long["bureau_vote"] == 3]

,bureau_vote,candidat,nb_voix
2,3,voix_liste_1,100
7,3,voix_liste_2,200
12,3,voix_liste_3,300
17,3,voix_liste_4,130


In [27]:
candidats_voix_long.shape

(20, 3)

In [28]:
# Vérification du nombre de candidats
candidats_voix_long.candidat.nunique()

4

### 3.3 - Jointure finale

In [29]:
data[cols_used["metrics_bdv"]].head()

,bureau_vote,type_election,annee,tour,commune,nb_inscrits,nb_abstentions,nb_votants,nb_emargements,nb_blanc,nb_nuls
0,1,MN,2025,1,48,931,131,800,800,50,50
1,2,MN,2025,1,48,1023,173,850,850,15,10
2,3,MN,2025,1,48,987,237,750,750,10,10
3,4,MN,2025,1,48,1036,236,800,800,10,10
4,5,MN,2025,1,48,1006,256,750,750,25,25


In [30]:
data[cols_used["metrics_bdv"]].info()

<class 'pandas.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   bureau_vote     32 non-null     int64
 1   type_election   32 non-null     str  
 2   annee           32 non-null     int64
 3   tour            32 non-null     int64
 4   commune         32 non-null     int64
 5   nb_inscrits     32 non-null     int64
 6   nb_abstentions  32 non-null     int64
 7   nb_votants      32 non-null     int64
 8   nb_emargements  32 non-null     int64
 9   nb_blanc        32 non-null     int64
 10  nb_nuls         32 non-null     int64
dtypes: int64(10), str(1)
memory usage: 2.9 KB


In [31]:
data_kpi = data[cols_used["metrics_bdv"]].copy()

In [32]:
candidats_voix_long.info()

<class 'pandas.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   bureau_vote  20 non-null     int64
 1   candidat     20 non-null     str  
 2   nb_voix      20 non-null     int64
dtypes: int64(2), str(1)
memory usage: 612.0 bytes


In [33]:
data_gold = candidats_voix_long.merge(
    data_kpi, 
    on="bureau_vote", 
    how="left"
)

In [34]:
data_gold

,bureau_vote,candidat,nb_voix,type_election,annee,tour,commune,nb_inscrits,nb_abstentions,nb_votants,nb_emargements,nb_blanc,nb_nuls
0,1,voix_liste_1,200,MN,2025,1,48,931,131,800,800,50,50
1,2,voix_liste_1,50,MN,2025,1,48,1023,173,850,850,15,10
2,3,voix_liste_1,100,MN,2025,1,48,987,237,750,750,10,10
3,4,voix_liste_1,150,MN,2025,1,48,1036,236,800,800,10,10
4,5,voix_liste_1,100,MN,2025,1,48,1006,256,750,750,25,25
5,1,voix_liste_2,300,MN,2025,1,48,931,131,800,800,50,50
6,2,voix_liste_2,300,MN,2025,1,48,1023,173,850,850,15,10
7,3,voix_liste_2,200,MN,2025,1,48,987,237,750,750,10,10
8,4,voix_liste_2,250,MN,2025,1,48,1036,236,800,800,10,10
9,5,voix_liste_2,200,MN,2025,1,48,1006,256,750,750,25,25
